In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import time

import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

import cv2

In [3]:
# read training images
DATADIR = "./train/"
CATEGORIES = ["Closed", "Open"]
IMG_SIZE = 100

training_data = []
for category in CATEGORIES:
        path = os.path.join(DATADIR, category) # path to eye image directory
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) # colour is not relevant in this use-case
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

random.shuffle(training_data)

X = [] # features
y = [] # labels

# label training data
for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # -1 is default, last 1 for grey scale
y = np.array(y)
y = to_categorical(y)

In [4]:
batch_size = 64
epochs = 20
num_classes = 4

# compare different architectures
dense_layers = [1]
layer_sizes = [16, 32]
conv_layers = [3, 5]

model = [Sequential() for i in range(len(dense_layers)*len(layer_sizes)*len(conv_layers))]
idx = 0

for dense_layer in dense_layers: 
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            # analyse model
            NAME = 'Drowsiness_Detection_Model_{}_{}_conv_{}_nodes_{}_dense_{}'.format(idx, conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            # input layer
            model[idx].add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model[idx].add(Activation('relu'))
            model[idx].add(MaxPooling2D(pool_size = (2,2))) 
            
            # hidden convulational layers (='filters')
            for l in range(conv_layer-1):
                model[idx].add(Conv2D(layer_size, (3,3)))
                model[idx].add(Activation('relu'))
                model[idx].add(MaxPooling2D(pool_size = (2,2)))
            
            # hidden dense (=fully connected) layers
            model[idx].add(Flatten())
            for l in range(dense_layer):
                model[idx].add(Dense(layer_size))
                model[idx].add(Activation('relu'))

            # output layer
            model[idx].add(Dense(len(CATEGORIES)))
            model[idx].add(Activation('sigmoid'))

            model[idx].compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

            model[idx].fit(X, y, batch_size=4, epochs=5, validation_split = 0.2) # appropriate batch size depends on size of data set
            idx+=1

2022-06-09 09:32:53.451335: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-09 09:32:53.451455: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro
Drowsiness_Detection_Model_0_3_conv_16_nodes_1_dense_1654759973
Epoch 1/5


2022-06-09 09:32:53.659174: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  1/291 [..............................] - ETA: 2:07 - loss: 0.9733 - accuracy: 0.5000

2022-06-09 09:32:53.925488: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

291/291 [==============================] - 3s 10ms/step - loss: 0.3950 - accuracy: 0.8932 - val_loss: 0.6159 - val_accuracy: 0.8729
Epoch 2/5
291/291 [==============================] - 3s 9ms/step - loss: 0.2324 - accuracy: 0.9225 - val_loss: 0.3586 - val_accuracy: 0.9175
Epoch 3/5
291/291 [==============================] - 3s 9ms/step - loss: 0.1497 - accuracy: 0.9569 - val_loss: 0.1956 - val_accuracy: 0.9175
Epoch 4/5
291/291 [==============================] - 3s 9ms/step - loss: 0.1133 - accuracy: 0.9604 - val_loss: 0.1471 - val_accuracy: 0.9725
Epoch 5/5
291/291 [==============================] - 3s 9ms/step - loss: 0.1323 - accuracy: 0.9569 - val_loss: 0.0969 - val_accuracy: 0.9656
Drowsiness_Detection_Model_3_5_conv_32_nodes_1_dense_1654760014
Epoch 1/5
  1/291 [..............................] - ETA: 2:25 - loss: 1.4035 - accuracy: 0.5000

2022-06-09 09:33:34.657632: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


291/291 [==============================] - ETA: 0s - loss: 0.3127 - accuracy: 0.8820

2022-06-09 09:33:37.662510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


291/291 [==============================] - 4s 11ms/step - loss: 0.3127 - accuracy: 0.8820 - val_loss: 0.2420 - val_accuracy: 0.9278
Epoch 2/5
291/291 [==============================] - 3s 10ms/step - loss: 0.1487 - accuracy: 0.9475 - val_loss: 0.0962 - val_accuracy: 0.9691
Epoch 3/5
291/291 [==============================] - 3s 10ms/step - loss: 0.1452 - accuracy: 0.9526 - val_loss: 0.0770 - val_accuracy: 0.9656
Epoch 4/5
291/291 [==============================] - 3s 10ms/step - loss: 0.0809 - accuracy: 0.9664 - val_loss: 0.1435 - val_accuracy: 0.9622
Epoch 5/5
291/291 [==============================] - 3s 10ms/step - loss: 0.1025 - accuracy: 0.9664 - val_loss: 0.0492 - val_accuracy: 0.9863


In [6]:
# save model
model[3].save('drowsinessEyeCnn.model')

INFO:tensorflow:Assets written to: drowsinessEyeCnn.model/assets


INFO:tensorflow:Assets written to: drowsinessEyeCnn.model/assets
